## Scenario 3: Multiple data scientists working on multiple ML models

MLflow setup:
* Tracking server: yes, remote server (EC2).
* Backend store: postgresql database.
* Artifacts store: s3 bucket.

The experiments can be explored by accessing the remote server.

The exampe uses AWS to host a remote server. In order to run the example you'll need an AWS account. Follow the steps described in the file [`mlflow-on-aws.md`](../mlflow-on-aws.md) to create a new AWS account and launch the tracking server. 

In [1]:
# Necessary import
import os, mlflow
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from mlflow.tracking import MlflowClient

# Set the environment - no need
os.environ["AWS_PROFILE"] = "magic-maxim" # fill in with your AWS profile. More info: https://docs.aws.amazon.com/sdk-for-java/latest/developer-guide/setup.html#setup-credentials

# AWS server host
TRACKING_SERVER_HOST = "ec2-34-227-105-189.compute-1.amazonaws.com" # fill in with the public DNS of the EC2 instance
# Set the tracking uri
mlflow.set_tracking_uri(f"http://{TRACKING_SERVER_HOST}:5000")

In [2]:
# Get the tracking uri
print(f"tracking URI: '{mlflow.get_tracking_uri()}'")

tracking URI: 'http://ec2-34-227-105-189.compute-1.amazonaws.com:5000'


In [3]:
# List experiments
mlflow.search_experiments() # list_experiments API has been removed, you can use search_experiments instead.()

[<Experiment: artifact_location='s3://mlflow-runs-artifacts-remote/0', creation_time=1748187695131, experiment_id='0', last_update_time=1748187695131, lifecycle_stage='active', name='Default', tags={}>]

## Creating an experiment and logging a new run

In [4]:
# Set a ML experiment
mlflow.set_experiment("my-remote-experiment")

# Start an experiment run 
with mlflow.start_run():
    # Load the iris data set features and target
    X, y = load_iris(return_X_y = True)

    # Model parameters
    params = {"C": 0.1, "random_state": 42}
    # Log the model parameters
    mlflow.log_params(params)

    # Initialize and train the logistic model
    lr = LogisticRegression(**params).fit(X, y)
    # Make predictions
    y_pred = lr.predict(X)
    # Log the model accuracy
    mlflow.log_metric("accuracy", accuracy_score(y, y_pred))

    # Log the model
    mlflow.sklearn.log_model(lr, artifact_path = "models")
    # Check the mlfow artiact uri
    print(f"default artifacts URI: '{mlflow.get_artifact_uri()}'")


2025/05/25 17:21:49 INFO mlflow.tracking.fluent: Experiment with name 'my-remote-experiment' does not exist. Creating a new experiment.
2025/05/25 17:21:53 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


default artifacts URI: 's3://mlflow-runs-artifacts-remote/2/aca1bc60ae77489e9d5b09aae848806c/artifacts'
🏃 View run carefree-shark-824 at: http://ec2-34-227-105-189.compute-1.amazonaws.com:5000/#/experiments/2/runs/aca1bc60ae77489e9d5b09aae848806c
🧪 View experiment at: http://ec2-34-227-105-189.compute-1.amazonaws.com:5000/#/experiments/2


In [5]:
# Show the updated lists of experiments
mlflow.search_experiments()

[<Experiment: artifact_location='s3://mlflow-runs-artifacts-remote/2', creation_time=1748190109625, experiment_id='2', last_update_time=1748190109625, lifecycle_stage='active', name='my-remote-experiment', tags={}>,
 <Experiment: artifact_location='s3://mlflow-runs-artifacts-remote/0', creation_time=1748187695131, experiment_id='0', last_update_time=1748187695131, lifecycle_stage='active', name='Default', tags={}>]

## Interacting with the model registry

In [6]:
# Create an mlflow client
client = MlflowClient(f"http://{TRACKING_SERVER_HOST}:5000")

In [7]:
# List registered models
client.search_registered_models()

[]

In [8]:
# Get the information about the first experiment run
run_id = client.search_runs(experiment_ids = '2')[0].info.run_id
# Register a model
mlflow.register_model(
    model_uri = f"runs:/{run_id}/models",
    name = 'iris-classifier'
)

Successfully registered model 'iris-classifier'.
2025/05/25 17:22:49 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: iris-classifier, version 1
Created version '1' of model 'iris-classifier'.


<ModelVersion: aliases=[], creation_timestamp=1748190170285, current_stage='None', description='', last_updated_timestamp=1748190170285, name='iris-classifier', run_id='aca1bc60ae77489e9d5b09aae848806c', run_link='', source='s3://mlflow-runs-artifacts-remote/2/aca1bc60ae77489e9d5b09aae848806c/artifacts/models', status='READY', status_message=None, tags={}, user_id='', version='1'>

---